pip install duckdb geopandas shapely pyarrow fsspec

In [10]:
import duckdb

con = duckdb.connect()

# 1. 필수 확장 기능 로드
con.execute("INSTALL spatial; LOAD spatial;")
con.execute("INSTALL httpfs; LOAD httpfs;")  #이건 AWS S3 접근을 위해 필요

In [11]:
# 3. 로컬 파일 경로 설정 (주피터 노트북과 같은 위치에 있다고 가정)
file_path = "US.parquet"

print(f"{file_path} 파일을 분석 중입니다...")

# 전체 데이터 개수 확인 
# read_parquet() 함수를 사용하여 로컬 파일을 직접 쿼리합니다.
total_count = con.execute(f"SELECT COUNT(*) FROM read_parquet('{file_path}');").fetchone()[0]
print(f"미국(US) 총 건물 수: {total_count:,}개")

# 5. 상위 5개 데이터 미리보기 
print("\n데이터 샘플 (상위 5개):")
sample_data = con.execute(f"SELECT * FROM read_parquet('{file_path}') LIMIT 5;").df()
print(sample_data)
# print(sample_data.head())

# 6. (선택) 특정 조건 필터링 예시: 면적이 100제곱미터 이상인 건물 수 
large_buildings = con.execute(f"""
    SELECT COUNT(*) 
    FROM read_parquet('{file_path}') 
    WHERE area_in_meters > 100;
""").fetchone()[0]
print(f"\n면적 100m² 초과 건물 수: {large_buildings:,}개")

US.parquet 파일을 분석 중입니다...
미국(US) 총 건물 수: 1,335,095개

데이터 샘플 (상위 5개):
   area_in_meters  confidence full_plus_code  \
0         15.1217      0.7597  8544JWR5+8RJR   
1          7.8240      0.6715  8544JWR6+979M   
2         15.4734      0.6743  8544JWR5+8QFQ   
3         18.3431      0.8013  8544JWR5+8PM4   
4         11.9122      0.6886  8544JWM8+QJVM   

                                            geometry       quadkey country_iso  
0  [2, 4, 0, 0, 0, 0, 0, 0, 89, 46, 234, 194, 51,...  023013221231          US  
1  [2, 4, 0, 0, 0, 0, 0, 0, 183, 45, 234, 194, 73...  023013221231          US  
2  [2, 4, 0, 0, 0, 0, 0, 0, 99, 46, 234, 194, 45,...  023013221231          US  
3  [2, 4, 0, 0, 0, 0, 0, 0, 117, 46, 234, 194, 48...  023013221231          US  
4  [2, 4, 0, 0, 0, 0, 0, 0, 191, 42, 234, 194, 18...  023013221231          US  

면적 100m² 초과 건물 수: 536,164개


## GeoPackage로 저장 

In [12]:
import pandas as pd
import geopandas as gpd
from shapely import wkb
from shapely.errors import GEOSException

# 지원되지 않는 WKB 타입을 건너뛰기 위해 TRY_CAST를 사용합니다.
con.execute("""
    CREATE OR REPLACE TABLE buildings_temp AS
    SELECT 
        * EXCLUDE (geometry), 
        TRY_CAST(geometry AS GEOMETRY) AS geom
    FROM read_parquet('US.parquet')
    WHERE geom IS NOT NULL  -- 변환에 성공한 데이터만 남김
    LIMIT 10000;
""")

# 2. GeoPackage 저장 (GDAL 드라이버 사용) 
con.execute("""
    COPY buildings_temp TO 'us_buildings.gpkg' 
    WITH (FORMAT GDAL, DRIVER 'GPKG');
""")

print("저장 완료: us_buildings.gpkg")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

저장 완료: us_buildings.gpkg


## PyDeck을 이용한 지도 시각화

In [13]:
import pydeck as pdk
import json

# 시각화를 위해 지오메트리를 WKT 문자열로 변환하여 가져오기
# 너무 많은 데이터는 브라우저 성능을 저하시키므로 LIMIT를 적절히 조절
query = """
    SELECT 
        area_in_meters,
        ST_AsGeoJSON(TRY_CAST(geometry AS GEOMETRY)) as geom_json
    FROM read_parquet('US.parquet')
    WHERE TRY_CAST(geometry AS GEOMETRY) IS NOT NULL
    LIMIT 15000;
"""
df = con.execute(query).df()

# 2. GeoJSON 문자열에서 좌표 리스트만 추출 (pydeck PolygonLayer 형식)
# Open Buildings는 보통 [[[...]]] 형태이므로 첫 번째 리스트([0])를 가져옵니다.
df['poly'] = df['geom_json'].apply(lambda x: json.loads(x)['coordinates'][0])

# 3. 데이터의 중심점 계산 (지도가 엉뚱한 곳을 비추지 않도록 설정)
sample_poly = df['poly'].iloc[0][0]
avg_lon, avg_lat = sample_poly[0], sample_poly[1] - 0.015  # 약간 남쪽으로 이동

# 4. Pydeck 3D 레이어 설정
layer = pdk.Layer(
    "PolygonLayer",
    df,
    get_polygon="poly",
    get_elevation= "area_in_meters / 20", # 면적을 높이로 활용 (Extrude) 
    get_fill_color="[140, 140, 240, 180]", # 오렌지색 3D 건물
    extruded=True,
    wireframe=True,
    pickable=True
)

# 5. 시각화 및 HTML 저장
view_state = pdk.ViewState(
    latitude=avg_lat, 
    longitude=avg_lon, 
    zoom=15, 
    pitch=65
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("3d_us_buildings.html")

print("3d_us_buildings.html 파일을 브라우저에서 열어보세요.")

3d_us_buildings.html 파일을 브라우저에서 열어보세요.


## 클라우드에서 바로 가져오기

In [19]:
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
# partitions = "by_country"
partitions = "by_country_s2"
country_iso = "USA"

# duckdb.sql(f"CREATE OR REPLACE TABLE lso_buildings AS SELECT * FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet') LIMIT 5000;")

con.execute(f"""
    CREATE OR REPLACE TABLE lso_buildings AS 
    SELECT * FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet') 
    LIMIT 5000;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
# 3. GeoPackage 저장 (GDAL/STRUCT/UNKNOWN Z 에러 해결)
# - EXCLUDE(geometry, bbox): GDAL이 처리 못하는 STRUCT 타입(bbox)과 원본 geometry 제외
# - TRY_CAST: 'UNKNOWN Z' 등 파싱 불가능한 지오메트리를 NULL로 처리하여 에러 방지
try:
    con.execute("""
        COPY (
            SELECT 
                * EXCLUDE (geometry, bbox), 
                TRY_CAST(geometry AS GEOMETRY) AS geom 
            FROM lso_buildings 
            WHERE geom IS NOT NULL
        ) 
        TO 'lso_buildings.gpkg' 
        WITH (FORMAT GDAL, DRIVER 'GPKG');
    """)
    print("성공: lso_buildings.gpkg 파일이 생성되었습니다.")
except Exception as e:
    print(f"저장 중 오류 발생: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

성공: lso_buildings.gpkg 파일이 생성되었습니다.


In [24]:
import duckdb
import folium
import pandas as pd
import json

# 3. Folium 시각화를 위해 GeoJSON 포맷으로 데이터 추출
# TRY_CAST를 사용하여 UNKNOWN Z 에러를 방지합니다.
query = """
    SELECT 
        area_in_meters,
        ST_AsGeoJSON(TRY_CAST(geometry AS GEOMETRY)) as geom_json
    FROM lso_buildings
    WHERE geometry IS NOT NULL
"""
df = con.execute(query).df()

# 4. Folium 지도 객체 생성 (데이터의 첫 번째 지점을 중심으로 설정)
first_geom = json.loads(df['geom_json'].iloc[0])
# GeoJSON은 [lon, lat] 순서이므로 folium용 [lat, lon]으로 뒤집기
center_lat, center_lon = first_geom['coordinates'][0][0][1], first_geom['coordinates'][0][0][0]

m = folium.Map(location=[center_lat, center_lon], zoom_start=17, tiles='CartoDB positron')

# 5. 데이터프레임의 각 행을 GeoJSON 레이어로 지도에 추가
for _, row in df.iterrows():
    geojson_data = json.loads(row['geom_json'])
    
    # 각 건물 폴리곤 추가
    folium.GeoJson(
        geojson_data,
        style_function=lambda x: {
            'fillColor': '#318ce7',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=f"Area: {row['area_in_meters']:.2f} m²"
    ).add_to(m)

# 6. HTML 파일로 저장
m.save("lso_buildings.html")
print("2D 시각화 완료: lso_buildings.html 파일을 확인하세요.")

2D 시각화 완료: lso_buildings.html 파일을 확인하세요.


In [26]:
# # 3. 지도 중심점 설정
# first_geom = json.loads(df['geom_json'].iloc[0])
# center_lat, center_lon = first_geom['coordinates'][0][0][1], first_geom['coordinates'][0][0][0]

# 4. 고해상도 위성 지도 설정 (Google Satellite)
m = folium.Map(location=[17.980962, -67.208852], zoom_start=18)

# 구글 위성 지도 타일 추가
google_satellite = folium.TileLayer(
    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr = 'Google',
    name = 'Google Satellite',
    overlay = False,
    control = True
).add_to(m)

# 5. 건물 폴리곤 추가
for _, row in df.iterrows():
    folium.GeoJson(
        json.loads(row['geom_json']),
        style_function=lambda x: {
            'fillColor': '#ffff00', # 위성 지도에서 잘 보이는 노란색
            'color': 'white',
            'weight': 1,
            'fillOpacity': 0.4
        },
        tooltip=f"Area: {row['area_in_meters']:.1f} m²"
    ).add_to(m)

m.save("lso_buildings2.html")
print("위성 영상 시각화 완료: lso_buildings2.html")

위성 영상 시각화 완료: lso_buildings2.html
